## Importing Needed Libraries

<p align="justify">insert spiel</p>

In [29]:
# Part 1: Data Preprocessing

import torch

import random

import numpy as np

from torch.utils.data import DataLoader, Subset

from torchvision import datasets, transforms

# Part 2: Creating the DCGAN Generator and Discriminator Classes

import torch.nn as nn

# Part 3: Training a DCGAN for Each Underrepresented Class (Cordana, Healthy, Pestalotiopsis)

import shutil

from pathlib import Path

from numpy import cos, pi

import torch.optim as optim

from torchvision.utils import save_image

import os

# https://docs.pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
# https://pyimagesearch.com/2021/10/25/training-a-dcgan-in-pytorch/
# https://medium.com/@manoharmanok/implementing-dcgan-in-pytorch-using-the-celeba-dataset-a-comprehensive-guide-660e6e8e29d2


## Part 1: Data Preprocessing

<p align="justify">insert spiel</p>

In [30]:
# constants

RAW_DATA_DIR = "../training_data"
GAN_SIZE = (128, 128)
CNN_SIZE = (224, 224)
BANANA_CLASSES  = ["cordana", "healthy", "pestalotiopsis", "sigatoka"]

<p align="justify">insert spiel</p>

In [31]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Ensure deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

<p align="justify">insert spiel</p>

In [32]:
transform_gan_b = transforms.Compose([
    transforms.Resize(GAN_SIZE), # Resize for DCGAN
    transforms.ToTensor(),       # To tensor
    transforms.Normalize(
        [0.5, 0.5, 0.5], 
        [0.5, 0.5, 0.5],
    )  # Normalize to [-1, 1] for DCGAN
])

transform_gan_p = transforms.Compose([
    transforms.RandomResizedCrop(GAN_SIZE, scale = (0.9, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomApply([ # maybe remove RandomApply for reproducibility?
        transforms.ColorJitter(
            brightness = 0.2, 
            contrast   = 0.2, 
            saturation = 0.2, 
            hue        = 0.05,
        ),
    ], p = 0.7),
    transforms.RandomApply([ # maybe remove RandomApply for reproducibility?
        transforms.RandomAffine(
            degrees   = 10, 
            translate = (0.1, 0.1), 
            scale     = (0.9, 1.0), 
        ),
    ], p = 0.7),
])

transform_cnn = transforms.Compose([
    transforms.Resize(CNN_SIZE), # Resize for CNN
    transforms.ToTensor(),       # To tensor
])

<p align="justify">insert spiel</p>

In [33]:
def load_gan_data(batch_size = 32, workers = 4, target_class = None, num_variants = 10, seed = 42, directory = RAW_DATA_DIR):

    generator = torch.Generator().manual_seed(seed)

    # Load full dataset with base GAN transformations
    dataset_gan = datasets.ImageFolder(root = directory, transform = transform_gan_b)

    if target_class:
        # Get class index from the class name
        class_index = dataset_gan.class_to_idx[target_class]

        # Filter indices where target matches
        indices = [i for i, (_, label) in enumerate(dataset_gan.samples) if label == class_index]

        # Wrap in a Subset
        dataset_gan = Subset(dataset_gan, indices)

        # Create a list to store augmented images
        augmented_images = []

        rng = random.Random(seed)

        # Apply augmentations to each image in the loaded dataset
        for i in range(len(dataset_gan)):
            image, label = dataset_gan[i]

            # Generate num_variants augmented versions of image
            for _ in range(num_variants):
                torch.manual_seed(rng.randint(0, 999999))
        
                augmented_image = transform_gan_p(image)

                augmented_images.append((augmented_image, label))

        # Create new dataset with augmented images
        final_dataset = torch.utils.data.TensorDataset(
            torch.stack([image for image, _ in augmented_images]),  # Stack all augmented images
            torch.tensor([label for _, label in augmented_images])  # Stack all labels
        )
    
    else:
        final_dataset = dataset_gan

    # Create DataLoader for the GAN data
    dataloader_gan = DataLoader(final_dataset, batch_size = batch_size, shuffle = True, num_workers = workers, generator = generator)

    return dataloader_gan

def load_cnn_data(batch_size = 32, workers = 4):
    # Load dataset with CNN transformations
    dataset_cnn = datasets.ImageFolder(root=RAW_DATA_DIR, transform = transform_cnn)
    
    # Create DataLoader for the CNN data
    dataloader_cnn = DataLoader(dataset_cnn, batch_size=batch_size, shuffle = True, num_workers = workers)

    return dataloader_cnn

## Part 2: Creating the DCGAN Generator and Discriminator Classes

<p align="justify">insert spiel</p>

In [34]:
# constants

SEED_NUM = 42            # Seed number for reproducibility

BATCH_SIZE = 128         # Number of images per training batch

INPUT_DIMENSION = 100    # Dimensionality of the generator input

NC = 3                   # Number of channels in the training images

NGF = 64                 # Base number of feature maps in the Generator

NDF = 64                 # Base number of feature maps in the Discriminator

EPOCHS = 200             # Number of training epochs

CHECKPOINT = 5           # Checkpoint number for model saving

LEARNING_RATE_G = 0.0002 # Learning rate for the Generator

LEARNING_RATE_D = 0.0001 # Learning rate for the Discriminator

BETA1 = 0.5              # Beta1 value for the Adam optimizer to help stabilize DCGAN training

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available

NGPU = 1  # Number of GPUs to use (0 means CPU only)

In [35]:
# Generator

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()

        self.ngpu = ngpu

        # Generator network composed of a stack of transposed conv blocks
        self.main = nn.Sequential(
            self._block(INPUT_DIMENSION, NGF * 16, 4, 1, 0, bias = False),  # First layer: latent vector -> feature map
            self._block(NGF * 16, NGF * 8, 4, 2, 1, bias = False),          # Upsample to 8 x 8
            self._block(NGF * 8, NGF * 4, 4, 2, 1, bias = False),           # Upsample to 16 x 16
            self._block(NGF * 4, NGF * 2, 4, 2, 1, bias = False),           # Upsample to 32 x 32
            self._block(NGF * 2, NGF, 4, 2, 1, bias = False),               # Upsample to 64 x 64

            nn.ConvTranspose2d(NGF, NC, 4, 2, 1, bias = False),             # Final upsample to 128 x 128 with RGB output
            nn.Tanh()                                                       # Output pixel values in [-1, 1]
        )

    # Helper function to define a generator block:

    # ConvTranspose2d -> InstanceNorm2d -> ReLU -> Dropout

    def _block(self, i_channels, o_channels, kernel_size, stride, padding, bias):
        return nn.Sequential(
            nn.ConvTranspose2d(
                i_channels, 
                o_channels, 
                kernel_size, 
                stride, 
                padding, 
                bias = bias),
            nn.InstanceNorm2d(o_channels, affine = True),
            nn.ReLU(True),
            nn.Dropout(0.3) # Dropout to help regularize on small data
        )

    def forward(self, input):
        return self.main(input)

In [36]:
# Discriminator

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()

        self.ngpu = ngpu

        # Discriminator network composed of downsampling conv blocks
        self.main = nn.Sequential(
            self._block(NC, NDF, 4, 2, 1, bias = False, use_instanceNorm2d = False), # First block: no InstanceNorm2d
            self._block(NDF, NDF *  2, 4, 2, 1, bias = False),                       # Downsample to 32 x 32
            self._block(NDF * 2, NDF *  4, 4, 2, 1, bias = False),                   # Downsample to 16 x 16
            self._block(NDF * 4, NDF *  8, 4, 2, 1, bias = False),                   # Downsample to 8 x 8
            self._block(NDF * 8, NDF * 16, 4, 2, 1, bias = False),                   # Downsample to 4 x 4

            nn.Conv2d(NDF * 16, 1, 4, 1, 0, bias = False),                           # Final layer: reduce to 1 x 1
        )

    # Helper function to define a discriminator block:

    # Conv2d -> (optional) InstanceNorm2d -> LeakyReLU

    def _block(self, i_channels, o_channels, kernel_size, stride, padding, bias, use_instanceNorm2d = True):
        layers = [nn.Conv2d(
            i_channels, 
            o_channels, 
            kernel_size, 
            stride, 
            padding, 
            bias = bias)]
        
        if use_instanceNorm2d:
            layers.append(nn.InstanceNorm2d(o_channels, affine = True))
        
        layers.append(nn.LeakyReLU(0.2, inplace = True))
        layers.append(nn.Dropout(0.3)) # Dropout to help regularize on small data

        return nn.Sequential(*layers)

    def forward(self, input):
        return self.main(input)

## Part 3: Training a DCGAN for Each Underrepresented Class (Cordana, Healthy, Pestalotiopsis)

<p align="justify">insert spiel</p>

In [37]:
def initialize_weights(model):
    classname = model.__class__.__name__

    if classname.find("Conv") != -1:
        nn.init.normal_(model.weight.data, 0.0, 0.02)

    elif classname.find("InstanceNorm") != -1:
        nn.init.normal_(model.weight.data, 1.0, 0.02)
        nn.init.constant_(model.bias.data, 0)

<p align="justify">insert spiel</p>

In [38]:
GAN_OUTPUT_DIRECTORY_TEST = "../model2/gan_test" # for debugging while training

def prepare_output_directory(resume = False):
    for cls in ["base"] + BANANA_CLASSES:
        if cls != "sigatoka":
            full_path = Path(GAN_OUTPUT_DIRECTORY_TEST) / cls

            # Only remove and recreate the directory if not resuming
            if not resume and full_path.exists():
                shutil.rmtree(full_path)

            full_path.mkdir(parents = True, exist_ok = True)

# prepare_output_directory(resume = True)

In [39]:
def train_dcgan(target_class = None, resume = False, checkpoint_path = None, balanced_path = None):
    # Set random seed for reproducibility
    set_seed(SEED_NUM)

    # Load dataset and define save path
    if balanced_path:  
        save_path = "../model2/final_dcgan"

        dataloader = load_gan_data(batch_size = BATCH_SIZE, directory = balanced_path)

    else:
        save_path = GAN_OUTPUT_DIRECTORY_TEST

        if target_class:
            dataloader = load_gan_data(batch_size = BATCH_SIZE, target_class = target_class)

        else:
            dataloader = load_gan_data(batch_size = BATCH_SIZE)

    # Initialize Generator and Discriminator
    netG = Generator(ngpu = NGPU).to(DEVICE)
    netD = Discriminator(ngpu = NGPU).to(DEVICE)

    # Handle multi-GPU setup if applicable
    if (DEVICE.type == "cuda") and (NGPU > 1):
        netG = nn.DataParallel(netG, list(range(NGPU)))
        netD = nn.DataParallel(netD, list(range(NGPU)))

    # Fixed noise for generating sample outputs and tracking progress during training
    fixed_generator = torch.Generator(device=DEVICE).manual_seed(SEED_NUM)

    fixed_noise = torch.randn(64, INPUT_DIMENSION, 1, 1, device = DEVICE, generator = fixed_generator)

    # Optimizers for Generator and Discriminator
    optimizerD = optim.Adam(netD.parameters(), lr = LEARNING_RATE_D, betas = (BETA1, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr = LEARNING_RATE_G, betas = (BETA1, 0.999))

    # Schedulers for optimizers
    schedulerD = torch.optim.lr_scheduler.CosineAnnealingLR(optimizerD, T_max = EPOCHS, eta_min = 1e-6)
    schedulerG = torch.optim.lr_scheduler.CosineAnnealingLR(optimizerG, T_max = EPOCHS, eta_min = 1e-6)

    # Default starting epoch
    start_epoch = 0

    # Loads checkpoint dcgan if provided 

    # Case 1: train base dcgan from a checkpoint
    # Case 2: train per-class dcgan from scratch with base dcgan weights as a starting point
    # Case 3: train per-class dcgan from a checkpoint
    if checkpoint_path and os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path, map_location = DEVICE)
        netG.load_state_dict(checkpoint["netG"])
        netD.load_state_dict(checkpoint["netD"])

        # Loads checkpoint optimizers if resuming training
        if resume:
            optimizerG.load_state_dict(checkpoint["optimizerG"])
            optimizerD.load_state_dict(checkpoint["optimizerD"])
            start_epoch = checkpoint["epoch"] + 1

    # Fresh start
    else:
        netG.apply(initialize_weights)
        netD.apply(initialize_weights)

    # Loss function
    criterion = nn.BCEWithLogitsLoss()

    # Labels for real and fake images
    real_label = 0.9 # Slightly less than 1
    fake_label = 0.1 # Slightly more than 0

    # Actual training
    for epoch in range(start_epoch, EPOCHS):
        # Calculate noise magnitude decay
        noise_magnitude = 0.1 * 0.5 * (1 + cos(pi * epoch / EPOCHS)) # 0.1 = maximum noise magnitude
    
        for i, (real_images, _) in enumerate(dataloader): # Iterate through batches in the dataset
            # 1. Update Discriminator: 
            #    maximize log(D(x)) + log(1 - D(G(z)))

            # 1.A. Train Discriminator on real images
            netD.zero_grad()

            # Format real batch
            real_images = real_images.to(DEVICE)

            # Well, train the Discriminator on noisy real images
            noise = torch.randn_like(real_images) * noise_magnitude # noise magnitude decays

            noisy_real_images = real_images + noise

            size = real_images.size(0)

            label = torch.full((size,), real_label, dtype = torch.float, device = DEVICE)

            # Forward pass noisy real images through Discriminator
            output = netD(noisy_real_images).view(-1)

            # Calculate Discriminator loss for noisy real images
            errD_real = criterion(output, label)

            # Backpropagate error for noisy real images
            errD_real.backward()

            # Mean output for noisy real images
            D_x = output.mean().item()

            # 1.B. Train Discriminator on batch of all fake images

            # Generate batch of latent vectors
            generator = torch.Generator(device=DEVICE).manual_seed(SEED_NUM + epoch)

            noise = torch.randn(size, INPUT_DIMENSION, 1, 1, device = DEVICE, generator = generator)

            # Generate fake images with Generator
            fake = netG(noise)

            # Classify fake images with Discriminator
            label.fill_(fake_label)

            # Forward pass fake images through Discriminator
            output = netD(fake.detach()).view(-1)

            # Calculate Discriminator loss for fake images
            errD_fake = criterion(output, label)

            # Backpropagate error for fake images
            errD_fake.backward()

            # Clip Discriminator gradients for stability
            torch.nn.utils.clip_grad_norm_(netD.parameters(), max_norm = 1.0)

            # Mean output for fake images
            D_G_z1 = output.mean().item()

            # Compute total Discriminator error = real error + fake error
            errD = errD_real + errD_fake

            # Finally update Discriminator
            optimizerD.step()

            # 2. Update Generator: 
            #    maximize log(D(G(z)))

            netG.zero_grad()
            label.fill_(real_label)  # fake labels are real for Generator cost

            # Pass fake images through Discriminator
            output = netD(fake).view(-1)

            # Calculate Generator loss based on Discriminator's output
            errG = criterion(output, label)

            # Backpropagate error for Generator
            errG.backward()

            # Mean output for fake images after Generator update
            D_G_z2 = output.mean().item()

            # Finally update Generator
            optimizerG.step()

            # Debugging: Print losses and monitor training progress

            if i in [0, len(dataloader)//2]:
                print(
                  f"Epoch [{epoch}/{EPOCHS}] Batch {i}/{len(dataloader)} \
                    Loss D: {errD.item():.4f}, loss G: {errG.item():.4f} \
                    D(x): {D_x:.4f}, \
                    D(G(z))_real: {D_G_z1:.4f}, D(G(z))_fake: {D_G_z2:.4f}"
                )

        # Step learning rate schedulers
        schedulerD.step()
        schedulerG.step()

        if epoch % CHECKPOINT == 0:
            fake_images = netG(fixed_noise).detach()

            if target_class:
                path = f"{save_path}/{target_class}"
            
            else:
                # root folder of save path, else /base if base dcgan
                path = f"{save_path}{"" if balanced_path else "/base"}"

            # Save images Generator could produce during checkpoints
            save_image(
                fake_images,
                os.path.join(path, f"sample_epoch_{epoch}.png"),
                normalize = True
            )

            # Save model version
            save_dict = {
                "epoch": epoch,
                "netG": netG.state_dict(),
                "netD": netD.state_dict(),
                "optimizerG": optimizerG.state_dict(),
                "optimizerD": optimizerD.state_dict(),
            }

            torch.save(save_dict, os.path.join(path, f"checkpoint_epoch_{epoch}.pth"))

    torch.save({
        "epoch": EPOCHS,
        "netG": netG.state_dict(),
        "netD": netD.state_dict(),
        "optimizerG": optimizerG.state_dict(),
        "optimizerD": optimizerD.state_dict(),
    }, os.path.join(f"{save_path}", f"{target_class if target_class else ("final" if balanced_path else "base")}_dcgan_final.pth"))

    return netG, netD

### Part 3.1: Training a Base DCGAN

<p align="justify">insert spiel</p>

In [40]:
# Train Base DCGAN

# trained_generator_base, trained_discriminator_base = train_dcgan(target_class = None, resume = False, checkpoint_path = None)

# trained_generator_base, trained_discriminator_base = train_dcgan(target_class = None, resume = True, checkpoint_path = '../model2/gan_test/base/checkpoint_epoch_10.pth')

### Part 3.2: Training a DCGAN Per Class

<p align="justify">insert spiel</p>

In [ ]:
# Train DCGAN for "cordana" class
# trained_generator_cordana, _ = train_dcgan(target_class = "cordana")

# trained_generator_cordana, _ = train_dcgan(target_class = "cordana", resume = True, checkpoint_path = "../model2/gan_test/cordana/checkpoint_epoch_<insert>.pth")

In [ ]:
# Train DCGAN for "healthy" class
# trained_generator_healthy, _ = train_dcgan(target_class = "healthy")

# trained_generator_healthy, _ = train_dcgan(target_class = "healthy", resume = True, checkpoint_path = "../model2/gan_test/healthy/checkpoint_epoch_<insert>.pth")

In [ ]:
# Train DCGAN for "pestalotiopsis" class
# trained_generator_pestalotiopsis, _ = train_dcgan(target_class = "pestalotiopsis")

# trained_generator_pestalotiopsis, _ = train_dcgan(target_class = "pestalotiopsis", resume = True, checkpoint_path = "../model2/gan_test/pestalotiopsis/checkpoint_epoch_<insert>.pth")

## Part 4: Generating Images for Each Underrepresented Class (Cordana, Healthy, Pestalotiopsis)

<p align="justify">insert spiel</p>

In [ ]:
def generate_synthetic_images(dcgan_generator, amount_to_generate, class_label, output_directory):

    # Set the generator to evaluation mode to disable Dropout and InstanceNorm2d updates
    dcgan_generator.eval()

    # Construct the path to the class-specific output directory
    class_output_directory = os.path.join(output_directory, class_label)

    # Create the output directory if it does not exist just in case
    os.makedirs(class_output_directory, exist_ok = True)

    # Disable gradient computation for efficiency during inference
    with torch.no_grad():
        for i in range(0, amount_to_generate, 16): # Batches of 16
            batch_size = min(16, amount_to_generate - i) # Adjusts batch size if near the end of generation

            # Sample random noise vectors as generator input
            noise = torch.randn(batch_size, INPUT_DIMENSION, 1, 1, device = DEVICE)

            # Generate a batch of fake images from the noise
            fake = dcgan_generator(noise)

            # Save each generated image to the output directory
            for j in range(batch_size):
                save_image(
                    fake[j], # Single image tensor
                    os.path.join(class_output_directory, f"gen_{i + j}.png"),
                    normalize = True
                )

In [ ]:
GAN_OUTPUT_DIRECTORY_BALANCED = "../model2/balanced"

# Target count based on the dominant Sigatoka class
TARGET_COUNT = 424

# Dictionary of class names and their real image counts
real_image_counts = {
    "cordana"        : 145,
    "healthy"        : 115,
    "pestalotiopsis" : 155,
}

# Dictionary mapping class labels to their corresponding trained generators
trained_generators = {
    "cordana"        : trained_generator_cordana,
    "healthy"        : trained_generator_healthy,
    "pestalotiopsis" : trained_generator_pestalotiopsis,
}

# Generate synthetic images for each underrepresented class
for label, real_count in real_image_counts.items():
    amount_to_generate = TARGET_COUNT - real_count

    generator = trained_generators[label]

    generate_synthetic_images(
        dcgan_generator = generator, amount_to_generate = amount_to_generate, class_label = label, output_directory = GAN_OUTPUT_DIRECTORY_BALANCED
    )

## Part 5: Training a DCGAN for feature extraction

<p align="justify">insert spiel</p>

In [ ]:
# train DCGAN on balanced training data

# balanced path is path to balanced training data
# save path of model is automatic to "../model2/final_dcgan"

trained_generator_final, trained_discriminator_final = train_dcgan(balanced_path = GAN_OUTPUT_DIRECTORY_BALANCED)